# Resume and CV Parsing with Spacy

Resume parsing is a process which converts an unstructured form of resume data into the structured format.

Resumes from the applicants have different formats in terms of presentation, design, fonts, and layouts. 

An ideal system should extract insightful information or the content inside these resumes as quickly as possible and help recruiters no matter how they look because they contain essential qualifications like the candidate's experience, skills, academic excellence.

## What is NER? 

Named Entity Recognition is an algorithm where it takes a string of text as an input (either a paragraph or sentence) and identifies relevant nouns (people, places, and organizations) and other specific words.

Followe Example Here: https://spacy.io/usage/training#training-data

## Data Preparation 

In [2]:
import spacy
import pickle
import random

In [3]:
train_data = pickle.load(open('train_data.pkl', 'rb'))

In [9]:
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

## NER with Spacy 

In [5]:
import warnings

In [12]:
nlp = spacy.blank('en')


def train_model(train_data):
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    #--------------------------
    
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
   
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        optimizer = nlp.begin_training()
            
        for itn in range(20):
            print('Starting iterations ', str(itn))
            random.shuffle(train_data)
            
            
            losses = {}
            index = 0
            for text, annotations in train_data:
                print(index,end = " ")
                index = index + 1
                try:
                    print(1)
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                    print(2)
                except Exception as e:
                    pass
#                     print(text)
                
            print(losses)




In [13]:
nlp.pipe_names

[]

In [14]:
train_model(train_data)

ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <spacy.pipeline.ner.EntityRecognizer object at 0x145f27280> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.

In [18]:
nlp.to_disk('nlp_model')

## Model Testing 

In [19]:
nlp_model = spacy.load('nlp_model')

In [22]:
text = train_data[0][0]

In [25]:
doc = nlp_model(text)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Vijayalakshmi Govindarajan
DESIGNATION                   - SAP as a Consultant
COMPANIES WORKED AT           - SAP Basis
LOCATION                      - Chennai
EMAIL ADDRESS                 - indeed.com/r/Vijayalakshmi-Govindarajan/ d71bfb70a66b0046
COMPANIES WORKED AT           - SAP Basis
DEGREE                        - MCA in Computer Applications
COLLEGE NAME                  - Thiagarajar School of Management
DEGREE                        - BSc
COLLEGE NAME                  - Sri Sathya Sai Institute of Higher Learning
DEGREE                        - HSC
COLLEGE NAME                  - TVS Lakshmi Matriculation Higher Secondary School
DEGREE                        - TVS Lakshmi Matriculation Higher Secondary School -
SKILLS                        - JAVA (6 years), ORACLE (6 years), SAP (6 years), ABAP (Less than 1 year), ACCESS (Less than 1 year)  ADDITIONAL INFORMATION  TECHNICAL EXPERTISE
COMPANIES WORKED AT           - SAP Basis
COMPANIES WORKED

## CV Parsing from PDF Data 

In [26]:
!pip install PyMuPDF
#tried PyPDF2 but does not work properly. 

In [27]:
import sys, fitz

In [28]:
fname = 'Alice Clark CV.pdf'
doc = fitz.open(fname)
text = ""

for page in doc:
    text = text + str(page.getText())

In [30]:
tx = " ".join(text.split('\n'))

In [31]:
tx

'Alice Clark  AI / Machine Learning    Delhi, India Email me on Indeed  •  20+ years of experience in data handling, design, and development  •  Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to  data warehousing and business intelligence  •  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, br

In [32]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Alice Clark
LOCATION                      - Delhi
COMPANIES WORKED AT           - Microsoft
DESIGNATION                   - Software Engineer
COMPANIES WORKED AT           - Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COMPANIES WORKED AT           - Microsoft
COLLEGE NAME                  - Indian Institute of Technology – Mumbai
SKILLS                        - SKILLS  Machine Learning, Natural Language Processing, and Big Data Handling    ADDITIONAL INFORMATION  Professional Skills  • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal  skills with ability to interact with individuals at all the levels  • Quick learner and maintains cordial relationship with project manager and team members and  good performer both in team